<a href="https://colab.research.google.com/github/MukhinaMaria/FinalAttest/blob/main/%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D0%B0%D1%8F_%D0%90%D1%82%D1%82%D0%B5%D1%81%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%D0%9C%D1%83%D1%85%D0%B8%D0%BD%D0%B0_%D0%9C_%D0%94_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import string

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("russian")
words=[]
classes = []
documents = []
ignore_words = ['?', '!', '«','»']
stop_words = set(stopwords.words('russian'))

data_file = open('/content/drive/MyDrive/datasets/questions.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words_lower = [word.lower() for word in words ]
# lemmatize, lower each word and remove duplicates
words = [stemmer.stem(w) for w in words_lower if w not in stop_words and w not in string.punctuation and w not in ignore_words]
#stemmer.stem(w)
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


122 documents
16 classes ['ФГАУ ФЦПР ИИ', 'задачи', 'заказать', 'платформа взаимодействия с экспертами', 'предложения', 'приветствие', 'прощание', 'решения', 'сайт', 'связаться', 'создан', 'специализированное ПО и прикладные сервисы для решения задач на базе ИИ', 'специализированные библиотеки', 'услуги', 'функционирует', 'цель']
56 unique stemmed words ['вечер', 'взаимодейств', 'вопрос', 'дня', 'добр', 'доступ', 'задач', 'заказа', 'здравств', 'знаеш', 'и', 'интеллект', 'искусствен', 'ка', 'как', 'коллективн', 'куп', 'межведомствен', 'минпромторг', 'моделирован', 'мпт', 'написа', 'ноч', 'обеспечен', 'ознаком', 'платформ', 'позвон', 'пок', 'получ', 'пользован', 'предлаагет', 'предлага', 'предлож', 'предложен', 'представител', 'привет', 'применен', 'программн', 'расскаж', 'решен', 'росс', 'сайт', 'связ', 'связа', 'созда', 'специализирова', 'так', 'технолог', 'услуг', 'утр', 'функицонир', 'хоч', 'цел', 'центр', 'цкп', 'эксперт']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-4-44ff79ea0aa3>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.keras', hist)

print("model created")

Epoch 1/200
25/25 [==============================] - 1s 5ms/step - loss: 2.7775 - accuracy: 0.1066
Epoch 2/200
25/25 [==============================] - 0s 5ms/step - loss: 2.5894 - accuracy: 0.2459
Epoch 3/200
25/25 [==============================] - 0s 4ms/step - loss: 2.5469 - accuracy: 0.1803
Epoch 4/200
25/25 [==============================] - 0s 5ms/step - loss: 2.3863 - accuracy: 0.2377
Epoch 5/200
25/25 [==============================] - 0s 6ms/step - loss: 2.1526 - accuracy: 0.2951
Epoch 6/200
25/25 [==============================] - 0s 3ms/step - loss: 1.9119 - accuracy: 0.4180
Epoch 7/200
25/25 [==============================] - 0s 3ms/step - loss: 1.8118 - accuracy: 0.3770
Epoch 8/200
25/25 [==============================] - 0s 3ms/step - loss: 1.6564 - accuracy: 0.4836
Epoch 9/200
25/25 [==============================] - 0s 4ms/step - loss: 1.5688 - accuracy: 0.5082
Epoch 10/200
25/25 [==============================] - 0s 7ms/step - loss: 1.3675 - accuracy: 0.5574
Epoch 11/

Добро пожаловать в чат-бот!

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.keras')
import json
import random
intents = json.loads(open('/content/drive/MyDrive/datasets/questions.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

# Start chatbot
while True:
    question = input('You: ')
    if question == 'quit':
        break
    answer = chatbot_response(question)
    print('Chatbot:', answer)


You: Привет
1/1 [==============================] - 0s 86ms/step
Chatbot: Доброго дня! Мы очень рады видеть Вас на этой страничке. Какие вопросы про Центр коллективного пользования Минпромторга России?
You: да
1/1 [==============================] - 0s 20ms/step
Chatbot: Спасибо, что обратились к нам!
You: центр
1/1 [==============================] - 0s 27ms/step
Chatbot: Центр коллективного пользования Минпромторга России «Межведомственная платформа моделирования и применения технологий искусственного интеллекта» является структурным подразделением Федерального государственного автономного учреждения «Федеральный центр прикладного развития искусственного интеллекта» 
You: по
1/1 [==============================] - 0s 20ms/step
Chatbot: Спасибо, что обратились к нам!
You: quit


Для заврешения работы чат-бота, введите "quit"


Для дополнительной оценки качества работы чат-бота и моделей, добавляю исходный текст, по которому составлялись наборы данных и обучался чат-бот:

Центр коллективного пользования Минпромторга России «Межведомственная платформа моделирования и применения технологий искусственного интеллекта» (далее – ЦКП МПТ ИИ, Центр, Центр коллективного пользования) – структурное подразделение ФГАУ «ФЦПР ИИ» (Федеральное государственное автономное учреждение «Федеральный центр прикладного развития искусственного интеллекта») создано с целью повышения доступности аппаратного обеспечении, необходимого для решения задач в области ИИ, а также апробации отечественных АПК.

ЦКП МПТ ИИ по направлению создания модели «Искусственный интеллект, как сервис», предоставляет следующие решения:

Специализированное ПО, прикладные сервисы для решения задач на базе ИИ, а именно:
Платформа для создания и разметки датасетов;
Нейросетевая библиотека.
Специализированные библиотеки, а именно:
Библиотека проверенных ML-моделей;
Библиотека обезличенных дата-сетов;
Платформа взаимодействия с экспертами.
https://aigov.ru/centre

